In [2]:
#프로젝트의 루트 디렉토리를 설정하기 위한 코드
import os
current_dir = os.getcwd()  # 현재 작업 디렉토리
print(os.path.dirname(current_dir))
# print(os.path.dirname(os.path.dirname(current_dir)))
# 루트 경로 설정 (예: 프로젝트 루트 경로)
project_root = os.path.dirname(current_dir)
os.chdir(project_root)  # 루트 경로로 변경

c:\sb-fsts\sb-fsts


In [3]:
from app.utils.crud_sql import SQLExecutor
from app.utils.database import get_db, get_db_session

sql_executor = SQLExecutor()

In [4]:
query = """
        SELECT 
            *
        FROM 
            fsts.test_table
    """

params = {}

with get_db_session() as db:
    result = sql_executor.execute_select(db, query, params)

print(result)

[{'id': 'id3', 'name': 'name3'}, {'id': 'id5', 'name': 'name5'}, {'id': 'id6', 'name': 'name6'}, {'id': 'id7', 'name': 'name7'}, {'id': 'id1', 'name': 'updated_name1'}, {'id': 'id2', 'name': 'upserted_name2'}, {'id': 'id4', 'name': 'upserted_name4'}]


In [5]:
# RETURNING * 꼭 붙여야 함.
query = """
    INSERT INTO fsts.test_table
    (id, name)
    VALUES (:id, :name)
    RETURNING *;
"""

id = 'id4'
name = 'name4'

params = {
    "id": id,
    "name": name
}

with get_db_session() as db:
    result = sql_executor.execute_insert(db, query, params)

HTTPException: 500: Duplicate Key error. already exists.

In [7]:
#여러 데이터를 한번에 삽입하고 싶을 때
# 여러 데이터 정의
data = [
    {"id": "id5", "name": "name5"},
    {"id": "id6", "name": "name6"},
    {"id": "id7", "name": "name7"}
]

# 다중 데이터 삽입 쿼리
query = """
    INSERT INTO fsts.test_table
    (id, name)
    VALUES (:id, :name)
    RETURNING *;
"""

# 데이터 삽입 및 결과 저장
results = []
with get_db_session() as db:
    for row in data:
        result = sql_executor.execute_insert(db, query, row)
        results.append(result)

print("삽입된 데이터:")
print(results)

HTTPException: 500: Duplicate Key error. already exists.

In [6]:
# UPDATE 테스트
query_update = """
    UPDATE fsts.test_table
    SET name = :name
    WHERE id = :id
    RETURNING *;
"""
#:는 파라미터 바인딩 형식

params_update = {
    "id": "id2",
    "name": "HONG"
}

with get_db_session() as db:
    result_update = sql_executor.execute_update(db, query_update, params_update)

Update succeeded: [{'id': 'id2', 'name': 'HONG'}]
